In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

FileNotFoundError: [Errno 2] No such file or directory: 'result/VGG16_quant/model_best.pth.tar'

In [8]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
42 -th layer prehooked
47 -th layer prehooked
51 -th layer prehooked
55 -th layer prehooked


In [4]:
weight_q = model.features[3].weight_q
w_alpha = model.features[3].weight_quant.wgt_alpha
w_bit = 4

weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))
print(weight_int)

tensor([[[[ 0.0000, -1.0000, -0.0000],
          [-1.0000,  0.0000, -1.0000],
          [-1.0000,  0.0000,  0.0000]],

         [[ 0.0000, -2.0000, -0.0000],
          [ 1.0000, -1.0000, -1.0000],
          [ 1.0000,  0.0000, -1.0000]],

         [[-0.0000, -2.0000,  0.0000],
          [ 0.0000, -2.0000, -2.0000],
          [ 1.0000, -1.0000, -1.0000]],

         ...,

         [[ 0.0000, -0.0000,  0.0000],
          [-1.0000, -2.0000, -2.0000],
          [ 0.0000, -2.0000, -1.0000]],

         [[-0.0000, -2.0000, -0.0000],
          [-0.0000, -1.0000, -1.0000],
          [ 0.0000, -1.0000, -1.0000]],

         [[ 0.0000, -1.0000, -0.0000],
          [-1.0000, -1.0000, -1.0000],
          [ 1.0000, -1.0000, -1.0000]]],


        [[[ 0.0000, -0.0000, -1.0000],
          [-2.0000, -2.0000, -2.0000],
          [-1.0000, -1.0000, -1.0000]],

         [[ 2.0000,  1.0000,  0.0000],
          [ 0.0000, -1.0000, -1.0000],
          [-0.0000, -2.0000, -2.0000]],

         [[ 0.0000,  1.0000, -0

In [9]:
act = save_output.outputs[1][0]
act_alpha  = model.features[3].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)

act_q = act_quant_fn(act, act_alpha)

act_int = act_q / (act_alpha / (2**act_bit-1))
print(act_int)

tensor([[[[0.0000, 1.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 2.0000],
          [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0000],
          [0.0000, 0.0000, 1.0000,  ..., 0

In [10]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, padding=1)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
conv_int.bias = model.features[3].bias
output_int = conv_int(act_int)
output_recovered = output_int * (act_alpha / (2**act_bit-1)) * (w_alpha / (2**(w_bit-1)-1))
print(output_recovered)

tensor([[[[-6.5186e+01, -8.7438e+01, -8.9318e+01,  ..., -8.0229e+01,
           -8.2737e+01, -7.8349e+01],
          [-5.5785e+01, -6.8320e+01, -7.1454e+01,  ..., -6.0799e+01,
           -6.2052e+01, -6.9261e+01],
          [-5.5785e+01, -6.6440e+01, -7.6155e+01,  ..., -6.6753e+01,
           -6.6753e+01, -7.2708e+01],
          ...,
          [-8.1796e+01, -9.9033e+01, -1.0091e+02,  ..., -9.0885e+01,
           -9.0258e+01, -8.8691e+01],
          [-7.8976e+01, -9.4332e+01, -9.6213e+01,  ..., -9.0258e+01,
           -8.6184e+01, -8.3050e+01],
          [-3.5100e+01, -3.9175e+01, -3.9175e+01,  ..., -4.1055e+01,
           -3.9801e+01, -4.2308e+01]],

         [[ 8.8064e+01,  1.2787e+02,  1.3006e+02,  ...,  1.1564e+02,
            1.2473e+02,  9.4332e+01],
          [ 1.0655e+02,  1.6453e+02,  1.6861e+02,  ...,  1.5607e+02,
            1.6829e+02,  1.2787e+02],
          [ 1.0499e+02,  1.6453e+02,  1.7268e+02,  ...,  1.5638e+02,
            1.7174e+02,  1.2975e+02],
          ...,
     

In [11]:
conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, padding=1)
conv_ref.weight = model.features[3].weight_q
conv_ref.bias = model.features[3].bias
output_ref = conv_ref(act)
print(output_ref)

tensor([[[[-6.7025e+01, -8.8101e+01, -9.1090e+01,  ..., -8.0171e+01,
           -8.3938e+01, -7.8862e+01],
          [-5.7119e+01, -6.9461e+01, -7.6328e+01,  ..., -6.4624e+01,
           -6.4877e+01, -7.1546e+01],
          [-5.7334e+01, -6.7483e+01, -7.6981e+01,  ..., -6.9992e+01,
           -7.0202e+01, -7.4934e+01],
          ...,
          [-8.0574e+01, -9.9076e+01, -1.0162e+02,  ..., -9.3072e+01,
           -9.1230e+01, -8.6107e+01],
          [-7.9103e+01, -9.4749e+01, -9.6193e+01,  ..., -9.4390e+01,
           -8.9447e+01, -8.2600e+01],
          [-3.5884e+01, -3.8560e+01, -4.0961e+01,  ..., -4.1927e+01,
           -4.0388e+01, -4.1751e+01]],

         [[ 8.7465e+01,  1.2795e+02,  1.3008e+02,  ...,  1.1595e+02,
            1.2578e+02,  9.3405e+01],
          [ 1.0827e+02,  1.6694e+02,  1.7138e+02,  ...,  1.5309e+02,
            1.6688e+02,  1.2750e+02],
          [ 1.0493e+02,  1.6461e+02,  1.7128e+02,  ...,  1.5714e+02,
            1.6931e+02,  1.2933e+02],
          ...,
     

In [16]:
# act_int.size = torch.Size([128, 64, 32, 32])  <- batch_size, input_ch, ni, nj
a_int = act_int[0,:,:,:]  # pick only one input out of batch
# a_int.size() = [64, 32, 32]

# conv_int.weight.size() = torch.Size([64, 64, 3, 3])  <- output_ch, input_ch, ki, kj
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij
# w_int.weight.size() = torch.Size([64, 64, 9])
                      
padding = 1
stride = 1
array_size = 64 # row and column number

nig = range(a_int.size(1))  ## ni group [0,1,...31]
njg = range(a_int.size(2))  ## nj group
 
icg = range(int(w_int.size(1)))  ## input channel [0,...63]
ocg = range(int(w_int.size(0)))  ## output channel


kijg = range(w_int.size(2)) # [0, .. 8]
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(njg)+padding*2).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))  ## mergin ni and nj index into nij
# a_pad.size() = [64, (32+2pad)*(32+2pad)]

In [17]:
###########################################

p_nijg = range(a_pad.size(1)) ## paded activation's nij group [0, ...34*34-1]

psum = torch.zeros( array_size, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:       
    for nij in p_nijg:     # time domain, sequentially given input
        m = nn.Linear(array_size, array_size, bias=False)
        m.weight = torch.nn.Parameter(w_int[:,:,kij])
        psum[:, nij, kij] = m(a_pad[:,nij]).cuda()
 

In [18]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32 + 2*pad = 34

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1) #34 - 2 - 1 + 1 = 32
o_nijg = range(o_ni_dim**2) # [0, 32*32-1]    
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:  #[0, ... 8]
        out[:,o_nij] = out[:,o_nij] + \
        psum[:, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 2nd index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

In [19]:
out_2D = torch.reshape(out, (out.size(0), o_ni_dim, -1)) # nij -> ni & nj
difference = (out_2D - output_int[0,:,:,:])
print(difference.abs().sum())

tensor(-0.3448, device='cuda:0', grad_fn=<SumBackward0>)


In [21]:
output_int[0,:,:,:]

tensor([[[-208.0000, -279.0000, -285.0001,  ..., -256.0000, -264.0000,
          -250.0000],
         [-178.0000, -218.0000, -228.0000,  ..., -194.0000, -198.0000,
          -221.0000],
         [-178.0000, -212.0000, -243.0000,  ..., -213.0000, -213.0000,
          -232.0000],
         ...,
         [-261.0000, -316.0000, -322.0000,  ..., -290.0000, -288.0000,
          -283.0000],
         [-252.0000, -301.0001, -307.0000,  ..., -288.0000, -275.0000,
          -265.0000],
         [-112.0000, -125.0000, -125.0000,  ..., -131.0000, -127.0000,
          -135.0000]],

        [[ 281.0000,  408.0000,  415.0000,  ...,  369.0000,  398.0000,
           301.0000],
         [ 340.0001,  525.0001,  538.0001,  ...,  498.0000,  537.0000,
           408.0000],
         [ 335.0000,  525.0001,  551.0000,  ...,  498.9999,  548.0000,
           414.0000],
         ...,
         [ 409.0000,  625.0001,  655.0001,  ...,  620.0000,  648.0000,
           487.0000],
         [ 419.0001,  631.0001,  650.000

In [ ]:
######## Easier 2D version ########

import math

kig = range(int(math.sqrt(len(kijg))))
kjg = range(int(math.sqrt(len(kijg))))
    
o_nig = range(int((math.sqrt(len(nijg))+2*padding -(math.sqrt(len(kijg))- 1) - 1)/stride + 1))
o_njg = range(int((math.sqrt(len(nijg))+2*padding -(math.sqrt(len(kijg)) - 1) - 1)/stride + 1))
    
    
out = torch.zeros(len(ocg), len(o_nig), len(o_njg)).cuda()
  
   
### SFP accumulation ###
for ni in o_nig:
    for nj in o_njg:
        for ki in kig:
            for kj in kjg:
                for ic_tile in ic_tileg:    
                    for oc_tile in oc_tileg:   
                        out[oc_tile*array_size:(oc_tile+1)*array_size, ni, nj] = out[oc_tile*array_size:(oc_tile+1)*array_size, ni, nj] + \
                        psum[ic_tile, oc_tile, :, int(math.sqrt(len(nijg)))*(ni+ki) + (nj+kj), len(kig)*ki+kj]